In [69]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import csv
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

In [70]:
with open('database_ampop.csv', 'r') as file:
    reader = csv.reader(file)
    header = next(reader)  # Lê a primeira linha como cabeçalho (opcional)
    data_base = [list(map(float, row)) for row in reader]

data_base = torch.tensor(data_base)
X_medidas = data_base[:,-3:].numpy()
Y_larguras = data_base[:,:-3].numpy()

print(len(X_medidas))
# print(Y_larguras[0,:])
print(X_medidas[0,:])

1006
[3.1064598e+01 6.1659500e+04 5.8053015e+06]


In [71]:
n_epochs = 1000
size_data_test = 0.2
hidden_size1 = 8
hidden_size2 = 8
learning_rate = 0.01
batch_size = 32

scaler_X = MinMaxScaler()
scaler_Y = MinMaxScaler()

X_normalized = scaler_X.fit_transform(X_medidas)
Y_normalized = scaler_Y.fit_transform(Y_larguras)

X_train, X_val, Y_train, Y_val = train_test_split(X_normalized, Y_normalized, test_size=size_data_test)
print(X_normalized[0,:])

X_train = torch.Tensor(X_train)
X_val = torch.Tensor(X_val)
Y_train = torch.Tensor(Y_train)
Y_val = torch.Tensor(Y_val)

train_dataset = TensorDataset(X_train, Y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size)

val_dataset = TensorDataset(X_val, Y_val)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

[0.01093262 0.00053323 0.34648305]


In [ ]:
class AmpOpT_NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_layer = nn.Linear(3,hidden_size1)
        self.h_layer1 = nn.Linear(hidden_size1,hidden_size2)
        self.h_layer2 = nn.Linear(hidden_size2,5)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.relu(self.input_layer(x))
        out = self.relu(self.h_layer1(out))
        out = self.h_layer2(out)
        return out

In [73]:
AmpOpT_pred_model = AmpOpT_NN()
loss = nn.MSELoss()

optimizer = torch.optim.Adam(AmpOpT_pred_model.parameters(), lr = learning_rate)

for epoch in range(n_epochs):
    AmpOpT_pred_model.train()
    train_loss = 0.0

    for X_batch, Y_batch in train_loader:
        output = AmpOpT_pred_model(X_train)
        l = loss(output, Y_train)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        train_loss += l.item()

    train_loss /= len(train_loader) 

    AmpOpT_pred_model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_val_batch, Y_val_batch in val_loader:
            val_pred = AmpOpT_pred_model(X_val_batch)
            val_loss += loss(val_pred, Y_val_batch).item()
    
    val_loss /= len(val_loader)
    
    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}/{n_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

Epoch 100/1000, Train Loss: 0.0535, Val Loss: 0.0509
Epoch 200/1000, Train Loss: 0.0516, Val Loss: 0.0482
Epoch 300/1000, Train Loss: 0.0515, Val Loss: 0.0480
Epoch 400/1000, Train Loss: 0.0511, Val Loss: 0.0483
Epoch 500/1000, Train Loss: 0.0511, Val Loss: 0.0483
Epoch 600/1000, Train Loss: 0.0511, Val Loss: 0.0483
Epoch 700/1000, Train Loss: 0.0511, Val Loss: 0.0483
Epoch 800/1000, Train Loss: 0.0511, Val Loss: 0.0483
Epoch 900/1000, Train Loss: 0.0511, Val Loss: 0.0483
Epoch 1000/1000, Train Loss: 0.0511, Val Loss: 0.0483


In [74]:
def predict(model, measures):
    model.eval()
    measures_normalized = scaler_X.transform([measures])
    measures_tensor = torch.tensor(measures_normalized, dtype=torch.float32)
    predictions = model(measures_tensor).detach().numpy()
    return scaler_Y.inverse_transform(predictions)

In [83]:
print('Ganho DC: Max =', max(X_medidas[:,0]), '; Min =', min(X_medidas[:,0]))
print('Freq Corte: Max =', max(X_medidas[:,1]), '; Min =', min(X_medidas[:,1]))
print('Slew Rate: Max =', max(X_medidas[:,2]), '; Min =', min(X_medidas[:,2]))

Ganho DC: Max = 2836.1963 ; Min = 0.058187954
Freq Corte: Max = 114815360.0 ; Min = 436.51584
Slew Rate: Max = 16696646.0 ; Min = 30904.166


In [87]:
novas_medidas = [2.5e3, 10e3, 15e6]
larguras_previstas = predict(AmpOpT_pred_model, novas_medidas)
print("Larguras de canal previstas:", larguras_previstas)

Larguras de canal previstas: [[6.48590049e-06 7.19695436e-06 7.50623167e-06 8.04592673e-06
  1.18833475e+01]]
